In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('reddit1.csv')

In [9]:
df.head(5)

,created_utc,ups,subreddit_id,link_id,name,score_hidden,author_flair_css_class,author_flair_text,subreddit,id,...,downs,archived,author,score,retrieved_on,body,distinguished,edited,controversiality,parent_id
0,1430438400,4,t5_378oi,t3_34di91,t1_cqug90g,0,NaN,NaN,soccer_jp,cqug90g,...,0,0,rx109,4,1432703079,くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな,NaN,0,0,t3_34di91
1,1430438400,4,t5_2qo4s,t3_34g8mx,t1_cqug90h,0,Heat,Heat,nba,cqug90h,...,0,0,WyaOfWade,4,1432703079,gg this one's over. off to watch the NFL draft...,NaN,0,0,t3_34g8mx
2,1430438400,0,t5_2cneq,t3_34f7mc,t1_cqug90i,0,NaN,NaN,politics,cqug90i,...,0,0,Wicked_Truth,0,1432703079,Are you really implying we return to those tim...,NaN,0,0,t1_cqufim0
3,1430438400,3,t5_2qh1i,t3_34f9rh,t1_cqug90j,0,NaN,NaN,AskReddit,cqug90j,...,0,0,jesse9o3,3,1432703079,No one has a European accent either because i...,NaN,0,0,t1_cqug2sr
4,1430438400,3,t5_2qh1i,t3_34fvry,t1_cqug90k,0,NaN,NaN,AskReddit,cqug90k,...,0,0,beltfedshooter,3,1432703079,"That the kid ""..reminds me of Kevin."" so sad...",NaN,0,0,t3_34fvry


In [10]:
df.columns

Index(['created_utc', 'ups', 'subreddit_id', 'link_id', 'name', 'score_hidden',
       'author_flair_css_class', 'author_flair_text', 'subreddit', 'id',
       'removal_reason', 'gilded', 'downs', 'archived', 'author', 'score',
       'retrieved_on', 'body', 'distinguished', 'edited', 'controversiality',
       'parent_id'],
      dtype='object')

In [55]:
word_count={}
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

for comments in df.body:
    no_punct = ""
    for char in comments.lower():
        if char not in punctuations:
             no_punct+=char
        if char == ' ':
            if no_punct in word_count:
                word_count[no_punct]+=1
            else:
                word_count[no_punct] = 1
            no_punct = ''

In [57]:
import operator
sorted_x = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)

In [77]:
from nltk.tokenize import word_tokenize

df['body'] = df['body'].astype('str')

comments = df['body']

word_count={}
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~=+'''

for comment in comments:
    for word in word_tokenize(comment.lower()):
        if word not in punctuations:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

In [89]:
import operator
sorted_x = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)

In [92]:
sorted_x

[('the', 1067389),
 ('to', 715329),
 ('a', 683159),
 ('', 631101),
 ('i', 597628),
 ('and', 579567),
 ('of', 456696),
 ('you', 407915),
 ('it', 378230),
 ('is', 374812),
 ('that', 366669),
 ('in', 344890),
 ('for', 272293),
 ('on', 197913),
 ('this', 197778),
 ('have', 191927),
 ('but', 191119),
 ('be', 184936),
 ('with', 182874),
 ('was', 177142),
 ('not', 171745),
 ('are', 164199),
 ('if', 152853),
 ('my', 152721),
 ('they', 141663),
 ('as', 141053),
 ('just', 137038),
 ('its', 133145),
 ('like', 131017),
 ('or', 128266),
 ('so', 127655),
 ('your', 121815),
 ('at', 114844),
 ('he', 109701),
 ('can', 101223),
 ('dont', 97361),
 ('me', 95007),
 ('what', 94868),
 ('all', 94838),
 ('do', 93173),
 ('would', 91778),
 ('im', 90934),
 ('get', 90169),
 ('about', 89870),
 ('one', 87055),
 ('from', 84509),
 ('up', 82132),
 ('out', 81921),
 ('an', 81676),
 ('more', 79277),
 ('we', 79217),
 ('people', 76849),
 ('no', 73892),
 ('will', 73025),
 ('when', 72967),
 ('there', 72352),
 ('think', 70551)